In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
from sqlite3 import IntegrityError
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
import csv
import requests 

In [ ]:
file_path_sdn = r'data/sdn.pip'
file_path_add = r'data/add.pip'
file_path_alt = r'data/alt.pip'
file_path_comments = r'data/sdn_comments.pip'
file_path_geocoded_id = r'data\gcr_coords_final.csv'

tables = ['SDN', 'add', 'alt', 'sdn_comments', 'geocoded_id']

columns_sdn = ['ent_num', 'SDN_name', 'SDN_Type', 'Program', 'Title', 'Call_Sign', 'Vess_type', 'Tonnage', 'GRT', 'Vess_Flag', 'Vess_owner', 'Remarks']
columns_add = ['ent_num', 'add_num', 'adress', 'csppc', 'country','add_remarks']
columns_alt = ['ent_num', 'alt_num', 'alt_type', 'alt_name', 'alt_remarks']
columns_comments = ['ent_num', 'remarks_extended']

columns_geocode_id = ['ent_num', 'lat', 'lng']

fp_columns = {file_path_sdn: columns_sdn,
              file_path_add: columns_add,
              file_path_alt: columns_alt,
          file_path_comments: columns_comments,
          file_path_geocoded_id: columns_geocode_id}

sdn_dataframes = [pd.read_csv(kvp[0], delimiter='|', header=None, names=kvp[1], na_values = ["-0-"]) for kvp in fp_columns.items()]

trunc_df = []
for df in sdn_dataframes:
   df = df[:-1]
   trunc_df.append(df)

load_dict = dict.fromkeys(tables, trunc_df)



In [ ]:
trunc_df[-1]

In [2]:
db_path = r'C:\Users\Juan\bootcamp\projects\project_2\dbase\us_sdn_final.db'
engine = create_engine(f'sqlite:///{db_path}')


In [ ]:
'''
for i in range(len(tables)-1):
    num_rows = len(trunc_df[i])
    #Iterate one row at a time
    for j in range(num_rows):
        try:
            #Try inserting the row
            trunc_df[i].iloc[j:j+1].to_sql(name=tables[i],con = engine ,if_exists = 'append', index=False)
        except IntegrityError:
            #Ignore duplicates
            pass
'''
for i in range(len(tables)-1):
    trunc_df[i].to_sql(tables[i], con=engine, if_exists='append', index=False)


In [ ]:
trunc_df[-1].to_sql(tables[-1], con=engine, if_exists='append', index=False)

In [15]:
geocode_path = r'C:\Users\Juan\bootcamp\projects\project_2\data\russian_addresses.csv'
adress_df = pd.read_csv(geocode_path)
print(adress_df.head())
country_codes = pd.read_csv(r'C:\Users\Juan\bootcamp\projects\project_2\data\positionstack-supported-countries.csv')
address_dict = adress_df.to_dict('records')

ent_num                               SDN_Name SDN_Type Program  add_num  \
0      173              ANGLO-CARIBBEAN CO., LTD.     -0-     CUBA      129   
1      735             EMPRESA CUBANA DE AVIACION     -0-     CUBA      441   
2      735             EMPRESA CUBANA DE AVIACION     -0-     CUBA      448   
3      751  EMPRESA CUBANA DE PESCADOS Y MARISCOS     -0-     CUBA      456   
4      767    ETCO INTERNATIONAL COMMODITIES LTD.     -0-     CUBA      465   

                                               query           region  \
0                           Ibex House, The Minories  London EC3N 1DY   
1                         Dobrininskaya No. 7, Sec 5           Moscow   
2  c/o Anglo-Caribbean Shipping Co. Ltd., Ibex Ho...  London EC3N 1DY   
3                                               -0-            Moscow   
4              Devonshire House, 1 Devonshire Street           London   

          country  
0  United Kingdom  
1          Russia  
2  United Kingdom  
3        

In [18]:
# This takes my API's supported country list and takes on the matching alpha2 code. NOTE: 
merged_cc = adress_df.merge(country_codes, on='country')
geocode_complete = merged_cc[['ent_num', 'SDN_Name', 'SDN_Type', 'Program','add_num', 'query', 'region', 'code', 'country']]
geocode_complete = geocode_complete.to_dict('records')

ECHNOSERVICES',
  'SDN_Type': '-0- ',
  'Program': 'CYBER2',
  'add_num': 37235,
  'query': 'd. 18 korp. 2 litera A, ul. Zheleznovodskaya',
  'region': 'St. Petersburg 199155',
  'code': 'RU',
  'country': 'Russia'},
 {'ent_num': 24648,
  'SDN_Name': 'TRIBUN, Aleksandr Lvovich',
  'SDN_Type': 'individual',
  'Program': 'CYBER2',
  'add_num': 37237,
  'query': '-0- ',
  'region': '-0- ',
  'code': 'RU',
  'country': 'Russia'},
 {'ent_num': 24650,
  'SDN_Name': 'DIGITAL SECURITY',
  'SDN_Type': '-0- ',
  'Program': 'CYBER2',
  'add_num': 37241,
  'query': '-0- ',
  'region': 'Saint Petersburg',
  'code': 'RU',
  'country': 'Russia'},
 {'ent_num': 24650,
  'SDN_Name': 'DIGITAL SECURITY',
  'SDN_Type': '-0- ',
  'Program': 'CYBER2',
  'add_num': 37242,
  'query': '-0- ',
  'region': 'Moscow',
  'code': 'RU',
  'country': 'Russia'},
 {'ent_num': 24651,
  'SDN_Name': 'KAGANSKIY, Vladimir Yakovlevich',
  'SDN_Type': 'individual',
  'Program': 'CYBER2',
  'add_num': 37243,
  'query': '-0- ',
 

In [20]:
## Starting over with the api call.
base_url = r'http://api.positionstack.com/v1/forward?access_key='
key = r'09a8d95720e74715ce80c81cf554891a'
# There are a few things getting in the way of collecting as many lat/long coordinate pairs. First, not all entries have complete records. We pre filtered all the ones where we couldn't get any info at all. This guarantees we will always have country level info for our query. So we have four possible outcomes collected into three logical statements:
#1 No query info (street adress level) AND No Region Info
#2 No query Info OR No Region Info
#3 Have all Info
geocoded_returns = []
i = 0
k = len(geocode_complete)
for address in geocode_complete:
    # This is going to get nasty: This bit deals with missing fields
    i +=1
    query_clause = f'{address["query"]}'
    region_clause = f'{address["region"]}'
    country_code_clause = f'&country={address["code"]}'
    optional_clause = '&country_module=1&limit=1'
    optional_country_clause = f'{address["country"]}'   
   
    if address["query"] == "-0- " and address["region"] == "-0- ":
        query_url = f'{base_url}{key}&query={optional_country_clause}{country_code_clause}{optional_clause}'
    
    elif address["query"] == "-0- " and address["region"] != "-0- ":
        query_url = f'{base_url}{key}&query={region_clause}{country_code_clause}{optional_clause}'
    
    elif address["query"] != "-0- " and address["region"] == "-0- ":
         query_url = f'{base_url}{key}&query={query_clause}{country_code_clause}{optional_clause}'

    else:
        query_url = f'{base_url}{key}&query={query_clause}&region={region_clause}{country_code_clause}{optional_clause}'
    

    print(f'Entity ID: {address["ent_num"]} :: Entity Country Code: {address["code"]}')
    returns = requests.get(query_url)
    
# This ugly bit trys another query to get any kind of data
 
    try:
        if len(returns.json()['data'])==0 and address['region']!='-0- ' and address['query']!='-0- ':
            query_url = f'{base_url}{key}&query={region_clause}{country_code_clause}{optional_clause}'
            returns = requests.get(query_url)
        elif len(returns.json()['data'])==0 and address['query']!='-0- ' and address['region']=='-0- ':
            query_url = f'{base_url}{key}&query={optional_country_clause}{country_code_clause}{optional_clause}'
            returns = requests.get(query_url)
            try:
                print(f'QOLR: adding records: {address["ent_num"]}')
                #print(query_url)
                geocoded_returns.append((address["ent_num"],address["SDN_Name"],address["SDN_Type"],address["Program"],returns.json()['data'][0]['latitude'],returns.json()['data'][0]['longitude']))
                print(f'{i} of {k}: Success!')
            except IndexError:
                print('Index Error!')
                print(query_url)
                pass
        else:
            try:
                print(f'Default Q: adding records: {address["ent_num"]}: {i} of {k}')
                if len(returns.json()['data'])!=0:
                    geocoded_returns.append((address["ent_num"],address["SDN_Name"],address["SDN_Type"],address["Program"],returns.json()['data'][0]['latitude'],returns.json()['data'][0]['longitude']))
                print(f'{i} of {k}: Success!')
                
            except IndexError:
                print('Index Error!')
                print(query_url)
                pass
    except KeyError:
        print('Key Error!')
        pass
        continue
    

: 22404 :: Entity Country Code: RU
Default Q: adding records: 22404: 305 of 508
305 of 508: Success!
Entity ID: 22405 :: Entity Country Code: RU
Default Q: adding records: 22405: 306 of 508
306 of 508: Success!
Entity ID: 22555 :: Entity Country Code: RU
Default Q: adding records: 22555: 307 of 508
307 of 508: Success!
Entity ID: 22556 :: Entity Country Code: RU
Default Q: adding records: 22556: 308 of 508
308 of 508: Success!
Entity ID: 19029 :: Entity Country Code: RU
Default Q: adding records: 19029: 309 of 508
309 of 508: Success!
Entity ID: 22417 :: Entity Country Code: RU
Default Q: adding records: 22417: 310 of 508
310 of 508: Success!
Entity ID: 22607 :: Entity Country Code: RU
Entity ID: 22607 :: Entity Country Code: RU
Entity ID: 22608 :: Entity Country Code: RU
Default Q: adding records: 22608: 313 of 508
313 of 508: Success!
Entity ID: 22610 :: Entity Country Code: RU
Default Q: adding records: 22610: 314 of 508
314 of 508: Success!
Entity ID: 22611 :: Entity Country Code: 

In [22]:
gcr_df = pd.DataFrame.from_records(geocoded_returns, columns=["ent_num","SDN_Name","SDN_Type","Program","lat","lng"])
geocoded_returns
gcr_df
gcr_df.to_json('data/russian_final.json', orient='values', index=True)
